# MODS Phenotypes - Step 1. Extract Data for Grady

## Imports

In [1]:
import pickle
from pathlib import Path
from tqdm import tqdm
import sys
import warnings
from random import sample
warnings.simplefilter(action="ignore", category=FutureWarning)
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from functools import reduce
import numpy as np

In [2]:
import sys
sys.path.insert(0, "/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/mods")
from src.config import *
from src.utils import *

In [3]:
site_name = 'grady'
sample_rate = 1

In [ ]:
output_path = (
    project_path / "data" / str(run_id) / "extraction" / site_name
)
output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
patient_id = project_config[site_name]["keys"]["patient_key"]
service_id = project_config[site_name]["keys"]["service_key"]
record_dt = project_config[site_name]["keys"]["record_dt"]

In [ ]:
# TODO: this shouldnt be needed make it go away
scores_keys = project_config[site_name]["scores"]
static_keys = project_config[site_name]["static"]
dynamic_keys = project_config[site_name]["dynamic"]
times_keys = project_config[site_name]["times"]
datetimes_keys = project_config[site_name]["datetimes"]

## Functions

### `extraction`

In [ ]:
def extraction(pickle_path):
    encounter_pickle, filename_without_ext = load_encounter_pickle(pickle_path)
    extract_dynamic_df(encounter_pickle, pickle_path, filename_without_ext)
    extract_static_df(encounter_pickle, pickle_path, filename_without_ext)
    extract_perCSN_dfs(encounter_pickle, pickle_path)

### `extract_dynamic_df`

In [ ]:
def extract_dynamic_df(encounter_pickle, pickle_path, filename_without_ext):
    super_df = get_super_df(encounter_pickle, pickle_path, filename_without_ext)
    
    scores_df = get_scores_df(encounter_pickle, pickle_path)
    
    dynamic_df = pd.merge(left=super_df, right=scores_df,
                          how='left', 
                        left_on=[record_dt], 
                        right_on=[record_dt],
                       suffixes=('_super','_dynamic'))

    dynamic_table = pa.Table.from_pandas(dynamic_df, preserve_index=False)
    
    output_folder = output_path / "dynamic_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)
    
    pq.write_table(dynamic_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   # TODO: Get from config
                   version='2.6', compression='snappy')

### `get_super_df`

In [ ]:
def get_super_df(encounter_pickle, pickle_path, filename_without_ext, cols_to_drop=None):
    # Get SuperTable
    try:
        super_df = encounter_pickle['super_table']
    except KeyError as e:
        print(f"(get_super_df) KeyError: 'super_table' not in {str(pickle_path.stem)}")
        return

    try:
        super_df[patient_id] = str(encounter_pickle['pt_id'])
    except KeyError as e:
        try:
            super_df[patient_id] = str(encounter_pickle['pat_id'])
        except KeyError as e:
            print(f"(get_super_df) KeyError: neither 'pt_id' or 'pat_id' was found in {str(pickle_path.stem)}")
            return

    # Drop columns
    if cols_to_drop:
        for col in cols_to_drop:
            try:
                super_df.drop(labels=col, axis=1, inplace=True)
            except KeyError:
                print(f"(get_super_df) KeyError in {str(pickle_path.stem)} when dropping '{col}' column")
                pass  # may not be in all encounters
    
    # Assign CSN to SuperTable
    try:
        str(encounter_pickle[service_id])
    except KeyError as e:
        super_df[service_id] = filename_without_ext  # Use the filename without extension here
    else:
        super_df[service_id] = str(encounter_pickle[service_id])

    super_df.reset_index(inplace=True, drop=False)
    super_df.rename(columns={"index": record_dt}, inplace=True)
    
    for key in (set(pandas_schema['dynamic']['super_table'].keys()) - set(super_df.columns)):
        super_df[key] = None
        super_df[key] = super_df[key].astype(pandas_schema['dynamic']['super_table'][key])

    try:
        super_df = super_df[pandas_schema['dynamic']['super_table'].keys()]
    except KeyError as e:
        print(f"(get_super_df) KeyError in {str(encounter_pickle[service_id])} when using keys from pandas_schema: {e}")

    super_schema = {}
    for col in super_df.columns:
        try:
            super_schema[col] = pandas_schema['dynamic']['super_table'][col]
        except KeyError as e:
            print(f"(get_super_df) KeyError with {col} in {str(pickle_path.stem)} when building super_schema")

    try:
        super_df = super_df.astype(super_schema)
    except TypeError as e:
        print(f"TypeError in {str(pickle_path.stem)}: {e}")
        # Identify and print problematic column and its unique values
        for col, dtype in super_schema.items():
            try:
                super_df[col].astype(dtype)
            except Exception as inner_e:
                print(f"Column {col} with values {super_df[col].unique()} caused error: {inner_e}")

    return super_df


#### `get_scores_df`

In [ ]:
def get_scores_df(encounter_pickle, pickle_path):
    # SOFA scores
    sofa_df = encounter_pickle['sofa_scores']
    sofa_rename_map = {
        'hourly_total': 'SOFA_hourly_total',
        'delta_24h': 'SOFA_delta_24h',
        'hourly_total_mod': 'SOFA_hourly_total_mod',
        'delta_24h_mod': 'SOFA_delta_24h_mod'
    }
    sofa_df.rename(columns=sofa_rename_map, inplace=True)

    # Check for 'sirs_scores' in encounter_pickle
    if 'sirs_scores' in encounter_pickle:
        sirs_df = encounter_pickle['sirs_scores']
        sirs_rename_map = {
            'hourly_total': 'SIRS_hourly_total',
            'delta_24h': 'SIRS_delta_24h'
        }
        sirs_df.rename(columns=sirs_rename_map, inplace=True)
    else:
        # Create an empty DataFrame with the same index as sofa_df
        sirs_df = pd.DataFrame(index=sofa_df.index)
    
    # Merging
    scores_df = pd.merge(left=sofa_df, right=sirs_df,
                         how='outer', left_index=True, right_index=True)

    # Type casting
    scores_schema = {}
    for key in scores_keys:
        # Check if key exists in scores_df
        if key in scores_df.columns:
            scores_schema = scores_schema | pandas_schema['dynamic']['scores'][key]

    # Check the existence of the column before trying to set its dtype
    for col, dtype in scores_schema.items():
        if col in scores_df.columns:
            try:
                scores_df[col] = scores_df[col].astype(dtype)
            except Exception as e:
                print(f"Error in {str(pickle_path.stem)} for column {col}: {e}")

    scores_df.reset_index(inplace=True, drop=False)
    scores_df.rename(columns={"index": record_dt}, inplace=True)
    
    return scores_df


### `extract_static_df`

In [ ]:
def extract_static_df(encounter_pickle, pickle_path, filename_without_ext):

    static_df = get_static_df(encounter_pickle)
    times_df = get_times_df(encounter_pickle)
    
    static_df = pd.concat([static_df, times_df], axis=1)

    static_schema = {}
    for key in static_keys:
        static_schema = static_schema | pandas_schema['static'][key]
    for key in times_keys:
        static_schema = static_schema | pandas_schema['static']['times'][key]

    # Filter the schema to only include columns present in the dataframe
    filtered_schema = {col: dtype for col, dtype in static_schema.items() if col in static_df.columns}
    try:
        static_df = static_df.astype(filtered_schema)
    except Exception as e:  # Catching a broader exception in case there are other issues beyond KeyError
        print(f"(extract_static_df) Error for {str(pickle_path.stem)} when doing .astype(filtered_schema): {e}")

    static_df.rename(columns={
        't_suspicion': 'times_suspicion_sepsis3',
        't_SOFA':'times_SOFA',
        't_sepsis3':'times_sepsis3',
        't_abx':'times_abx_order',
        't_clt':'times_culture'
        },
                     inplace=True)
    # dealing with `ed_wait_time`
    if type(static_df.loc[0,'ed_wait_time']) is pd.Timedelta:
        static_df.loc[0,'ed_wait_time'] = float(static_df['ed_wait_time'][0].seconds/60)

    if pd.isnull(static_df.loc[0,'ed_wait_time']):
        static_df.loc[0,'ed_wait_time'] = 0.0
        static_df.loc[0,'ed_wait_time'] = float('nan')            

    static_table = pa.Table.from_pandas(static_df, preserve_index=False)

    output_folder = output_path / "static_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)

    pq.write_table(static_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   # TODO: Get from config
                   version='2.6', compression='snappy')

### `get_static_df`

In [ ]:
def get_static_df(encounter_pickle):
    static_dict = reduce(lambda a, b: {**a, **b}, [encounter_pickle[k] for k in static_keys])

    # Dealing with ed_wait_time issues
    if 'ed_wait_time' not in static_dict.keys():
        static_dict['ed_wait_time'] = float('nan')
    
    static_df = pd.DataFrame(pd.Series(static_dict)).T
    return static_df

### `get_times_df`

In [ ]:
def get_times_df(encounter_pickle):
    times_data = [
        denoise_times(encounter_pickle['sep3_time'].t_suspicion),
        denoise_times(encounter_pickle['t_suspicion'].t_clt),
        denoise_times(encounter_pickle['t_suspicion'].t_abx),
        denoise_times(encounter_pickle['sep3_time'].t_SOFA),
        denoise_times(encounter_pickle['sep3_time'].t_sepsis3)
    ]

    times_df = pd.DataFrame([times_data], columns = [
        't_suspicion',
        't_clt',
        't_abx',
        't_SOFA',
        't_sepsis3'
        ])
    
    for n in times_df.columns:
        try:
            if times_df[n][0].size == 0:
                times_df[n][0] = [pd.NaT]
        except:
            pass

    return times_df

### `extract_perCSN_dfs`

In [ ]:
def extract_perCSN_dfs(encounter_pickle, pickle_path):
    extract_beds_df(encounter_pickle, pickle_path)
    extract_diagnosis_df(encounter_pickle, pickle_path)
    extract_procedures_df(encounter_pickle, pickle_path)
    extract_cultures_df(encounter_pickle, pickle_path)

In [ ]:
def extract_beds_df(encounter_pickle, pickle_path):
    beds_df = encounter_pickle["beds_PerCSN"]
    beds_df.reset_index(inplace=True)
    beds_df.rename(columns={'index':'csn'}, inplace=True)

    beds_table = pa.Table.from_pandas(beds_df, preserve_index=False)

    output_folder = output_path / "beds_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)

    pq.write_table(beds_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   version='2.6', compression='snappy')

In [ ]:
def extract_diagnosis_df(encounter_pickle, pickle_path):
    diagnosis_df = encounter_pickle["diagnosis_PerCSN"]
    diagnosis_df.reset_index(inplace=True)
    diagnosis_df.rename(columns={'index':'csn'}, inplace=True)

    diagnosis_table = pa.Table.from_pandas(diagnosis_df, preserve_index=False)

    output_folder = output_path / "diagnosis_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)

    pq.write_table(diagnosis_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   version='2.6', compression='snappy')

In [ ]:
def extract_procedures_df(encounter_pickle, pickle_path):
    procedures_df = encounter_pickle["procedures_PerCSN"]
    procedures_df.reset_index(inplace=True)
    procedures_df.rename(columns={'index':'csn'}, inplace=True)

    procedures_table = pa.Table.from_pandas(procedures_df, preserve_index=False)

    output_folder = output_path / "procedures_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)

    pq.write_table(procedures_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   version='2.6', compression='snappy')

In [ ]:
def extract_cultures_df(encounter_pickle, pickle_path):
    cultures_df = encounter_pickle["cultures_PerCSN"]
    cultures_df.reset_index(inplace=True)
    cultures_df.rename(columns={'index':'csn'}, inplace=True)

    cultures_table = pa.Table.from_pandas(cultures_df, preserve_index=False)

    output_folder = output_path / "cultures_df" / str(pickle_path.parent.stem)
    output_folder.mkdir(parents=True, exist_ok=True)

    pq.write_table(cultures_table,
                   output_folder / f"{pickle_path.stem}.parquet",
                   version='2.6', compression='snappy')

## `main()`

In [ ]:
def main(config):
    file_path=config[site_name]['filepaths']['encounter_pickles']
    years=config[site_name]['years']
    sample_rate=config['parameters']['sample_rate']
    num_cpus=config['parameters']['num_cpus']
    pickle_paths = find_pickle_paths(file_path=file_path,
                                     years=years,
                                     sample_rate=sample_rate)
    with Pool(processes=num_cpus) as pool:
        max_ = len(pickle_paths)
        with tqdm(total=max_) as pbar:
            for _ in pool.imap_unordered(func=extraction, iterable=pickle_paths):
                pbar.update()

main(project_config)

---
---
---

## Load results

### Load `dynamic_df`

In [7]:
%%time
dynamic_schema = (
    arrow_schema['dynamic']['super_table'] |
    reduce(lambda a, b: {**a, **b}, [arrow_schema['dynamic']['scores'][k] for k in scores_keys])
    )

arrow_dynamic_schema = make_arrow_schema(dynamic_schema)

dynamic_table = pq.read_table(
    str(output_path/'dynamic_df'/'2018'/'1021009657.parquet'),
    use_pandas_metadata=True,
    schema=arrow_dynamic_schema
)
dynamic_df = dynamic_table.to_pandas()
display(dynamic_df.head())

,pat_id,csn,charttime,temperature,daily_weight_kg,height_cm,sbp_line,dbp_line,map_line,sbp_cuff,...,SOFA_hourly_total,SOFA_delta_24h,SOFA_hourly_total_mod,SOFA_delta_24h_mod,SIRS_resp,SIRS_cardio,SIRS_temp,SIRS_wbc,SIRS_hourly_total,SIRS_delta_24h
0,Z2128263,1021009657,2018-02-26 12:06:00,100.6,49.9,165.1,NaN,NaN,NaN,144.5,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,3.0
1,Z2128263,1021009657,2018-02-26 13:06:00,100.6,49.9,165.1,NaN,NaN,NaN,156.5,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0
2,Z2128263,1021009657,2018-02-26 14:06:00,103.1,49.9,165.1,NaN,NaN,NaN,165.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0
3,Z2128263,1021009657,2018-02-26 15:06:00,101.5,49.9,165.1,NaN,NaN,NaN,172.0,...,2.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,3.0,4.0
4,Z2128263,1021009657,2018-02-26 16:06:00,101.5,49.9,165.1,NaN,NaN,NaN,172.0,...,2.0,2.0,2.0,2.0,0.0,1.0,1.0,1.0,3.0,4.0


CPU times: user 62.8 ms, sys: 31.2 ms, total: 94 ms
Wall time: 305 ms


### Load `static_df`

In [1]:
%%time
static_schema = (
    reduce(lambda a, b: {**a, **b}, [arrow_schema['static'][k] for k in static_keys])
    |
    reduce(lambda a, b: {**a, **b}, [arrow_schema['static']['times'][k] for k in times_keys])
    )
static_schema['times_abx_order'] = 'LIST(TIMESTAMP[NS])'
static_schema['times_culture'] = 'LIST(TIMESTAMP[NS])'
static_schema['times_suspicion_sepsis3'] = 'LIST(TIMESTAMP[NS])'
static_schema['times_SOFA'] = 'LIST(TIMESTAMP[NS])'
static_schema['times_sepsis3'] = 'LIST(TIMESTAMP[NS])'

arrow_static_schema = make_arrow_schema(static_schema)

static_table = pq.read_table(
    str(output_path/'static_df'/'2018'/'1021009657.parquet'),
    use_pandas_metadata=True,
    schema=arrow_static_schema
)

static_df = static_table.to_pandas()

display(static_df.head())

NameError: name 'reduce' is not defined

### Load `beds_PerCSN`

In [9]:
arrow_schema['perCSN']['beds_PerCSN']['csn']='STRING'
arrow_schema['perCSN']['beds_PerCSN']['pat_id']='STRING'

In [10]:
%%time
arrow_beds_schema = make_arrow_schema(arrow_schema['perCSN']['beds_PerCSN'])

beds_table = pq.read_table(
    str(output_path/'beds_df'/'2018'/'1021009657.parquet'),
    use_pandas_metadata=True,
    schema=arrow_beds_schema
)

beds_df = beds_table.to_pandas()

display(beds_df.head())

,csn,pat_id,bed_location_start,bed_location_end,bed_unit,bed_room,bed_id,bed_label,hospital_service,accomodation_code,accomodation_description,icu,imc,ed,procedure
0,1021009657,Z2128263,2018-02-26 12:10:00,2018-02-26 14:36:00,GHS EMERGENCY,Z2-20,7053,Z2-20,Emergency Medicine (Non-admitting),10016,Intermediate Care,0.0,0.0,1.0,0.0
1,1021009657,Z2128263,2018-02-26 14:36:00,2018-02-27 03:47:00,GHS EMERGENCY,Z3-47,7199,Z3-47,Emergency Medicine (Non-admitting),10016,Intermediate Care,0.0,0.0,1.0,0.0
2,1021009657,Z2128263,2018-02-27 03:47:00,2018-02-27 04:05:00,GHS EMERGENCY,OTF1,1001,OTF,Emergency Medicine (Non-admitting),10016,Intermediate Care,0.0,0.0,1.0,0.0
3,1021009657,Z2128263,2018-02-27 04:05:00,2018-02-28 21:28:00,GHS 5KOF,5KOF,7405,5K09,Emergency Medicine (Non-admitting),10016,Intermediate Care,0.0,0.0,0.0,0.0


CPU times: user 14.3 ms, sys: 11.8 ms, total: 26.1 ms
Wall time: 74.1 ms


### Load `cultures_PerCSN`

In [11]:
%%time
arrow_schema['perCSN']['cultures_PerCSN']['csn']='STRING'
arrow_schema['perCSN']['cultures_PerCSN']['pat_id']='STRING'
arrow_schema['perCSN']['cultures_PerCSN']['proc_code']='STRING'
arrow_schema['perCSN']['cultures_PerCSN']['component_id']='STRING'
arrow_schema['perCSN']['cultures_PerCSN']['order_id']='STRING'
arrow_schema['perCSN']['cultures_PerCSN']['result_id']='STRING'
arrow_cultures_schema = make_arrow_schema(arrow_schema['perCSN']['cultures_PerCSN'])

cultures_table = pq.read_table(
    str(output_path/'cultures_df'/'2018'/'1021009657.parquet'),
    use_pandas_metadata=True,
    schema=arrow_cultures_schema
)

cultures_df = cultures_table.to_pandas()

display(cultures_df.head())

,csn,pat_id,proc_code,proc_desc,component_id,component,loinc_code,specimen_collect_time,order_time,order_id,result_id,lab_result_time,result_status,lab_result
0,1021009657,Z2128263,LAB239,URINE CULTURE,123040764,Culture,11475-1,2018-02-27 03:01:00,2018-02-26 19:14:00,139732207,P389850,2018-02-28 10:51:00,Preliminary,"10-100,000 CFU/ML Escherichia coli"
1,1021009657,Z2128263,LAB239,URINE CULTURE,123040764,Culture,11475-1,2018-02-27 03:01:00,2018-02-26 19:14:00,139732207,P392144,2018-02-28 13:29:00,Preliminary,"10-100,000 CFU/ML Escherichia coli"
2,1021009657,Z2128263,LAB239,URINE CULTURE,123040764,Culture,11475-1,2018-02-27 03:01:00,2018-02-26 19:14:00,139732207,P375048,2018-03-01 10:15:00,Verified,"10-100,000 CFU/ML Escherichia coli"
3,1021009657,Z2128263,LAB462,BLOOD CULTURE,123040764,Culture,11475-1,2018-02-26 20:26:00,2018-02-26 19:35:00,139732211,P384181,2018-02-27 21:00:00,Preliminary,Escherichia coli
4,1021009657,Z2128263,LAB462,BLOOD CULTURE,123040764,Culture,11475-1,2018-02-26 20:26:00,2018-02-26 19:35:00,139732211,P384181,2018-02-27 21:00:00,Preliminary,Critical Results


CPU times: user 18.5 ms, sys: 5.43 ms, total: 23.9 ms
Wall time: 17.4 ms


### Load `procedures_PerCSN`

In [12]:
%%time
arrow_schema['perCSN']['procedures_PerCSN']['csn']='STRING'
arrow_schema['perCSN']['procedures_PerCSN']['pat_id']='STRING'
arrow_procedures_schema = make_arrow_schema(arrow_schema['perCSN']['procedures_PerCSN'])

procedures_table = pq.read_table(
    str(output_path/'procedures_df'/'2018'/'1021729214.parquet'),
    use_pandas_metadata=True,
    schema=arrow_procedures_schema
)

procedures_df = procedures_table.to_pandas()

display(procedures_df.head())

,csn,pat_id,surgery_date,in_or_dttm,procedure_start_dttm,procedure_comp_dttm,out_or_dttm,or_procedure_id,primary_procedure_nm,cpt_code,service_nm,primary_physician_nm
0,1021729214,Z2202010,2018-03-02,2018-03-02 07:56:00,2018-03-02 08:24:00,2018-03-02 10:42:00,2018-03-02 10:57:00,None,TANGENTIAL EXCISION OF BURN WOUND,None,Burns,"WILLIAMS, RACHAEL Y."
1,1021729214,Z2202010,2018-03-16,2018-03-16 07:46:00,2018-03-16 08:13:00,2018-03-16 08:56:00,2018-03-16 09:06:00,None,"AMPUTATION, TOE; METATARSOPHALANGEAL JOINT",28820,Burns,"WILLIAMS, RACHAEL Y."
2,1021729214,Z2202010,2018-04-12,2018-04-12 12:59:00,2018-04-12 13:46:00,2018-04-12 18:31:00,2018-04-12 18:31:00,None,TANGENTIAL EXCISION OF BURN WOUND,None,Burns,"INGRAM, WALTER L."


CPU times: user 13.6 ms, sys: 7.46 ms, total: 21.1 ms
Wall time: 17.2 ms


### Load `diagnosis_PerCSN`

In [13]:
%%time
arrow_schema['perCSN']['diagnosis_PerCSN']['csn']='STRING'
arrow_schema['perCSN']['diagnosis_PerCSN']['pat_id']='STRING'
arrow_diagnosis_schema = make_arrow_schema(arrow_schema['perCSN']['diagnosis_PerCSN'])

diagnosis_table = pq.read_table(
    str(output_path/'diagnosis_df'/'2018'/'1021735826.parquet'),
    use_pandas_metadata=True,
    schema=arrow_diagnosis_schema
)

diagnosis_df = diagnosis_table.to_pandas()

display(diagnosis_df.head())

,csn,pat_id,dx_line,dx_icd_scope,dx_code_icd9,dx_code_icd10,dx_source,dx_time_date,dx_code,dx_name
0,1021735826,Z2172494,1,None,None,O365930,ADMISSION DX (CODED),2018-02-08 06:21:40,O36.5930,Maternal care for other known or suspected poo...
1,1021735826,Z2172494,1,Yes,None,O365930,FINAL DIAGNOSES,2018-02-08 06:21:00,O36.5930,Maternal care for other known or suspected poo...
2,1021735826,Z2172494,2,Yes,None,R030,FINAL DIAGNOSES,2018-02-08 06:21:00,R03.0,"Elevated blood-pressure reading, without diagn..."
3,1021735826,Z2172494,3,Yes,None,O9989,FINAL DIAGNOSES,2018-02-08 06:21:00,O99.89,Other specified diseases and conditions compli...
4,1021735826,Z2172494,4,Exempt from POA reporting,None,Z370,FINAL DIAGNOSES,2018-02-08 06:21:00,Z37.0,Single live birth


CPU times: user 16 ms, sys: 3.77 ms, total: 19.8 ms
Wall time: 47.8 ms
